Name: **Luong Nguyen**  
Student ID: **1504210**  

# Introduction to Deep Learning 

## Session07: word-level text generation
____
Based on example code from Chollet's book chapter [8.1](https://github.com/fchollet/deep-learning-with-python-notebooks/blob/master/8.1-text-generation-with-lstm.ipynb)

## Import the packages needed for this assignment

In [2]:
# install keras on google colab
!pip install -q keras
# is GPU working?
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [3]:
# import packages
import numpy as np
import keras
from keras import models, layers
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


## Prepare data

### Load data

I downloaded 10 Metropolia Bsc theses written in English related to Web programming from Theseus, converted them to .txt, concat them into 1 .txt file.

In [4]:
path = keras.utils.get_file(
    'text.txt',
    origin='http://users.metropolia.fi/~luongn/text.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))

Corpus length: 656120


A look at `text`

In [44]:
text[:100]

'befekadu mezgebu temere\n\nresponsive web application using bootstrap\nand foundation\ncomparing bootstr'

### Preprocess data

Replace non-alphanumeric characters with whitespaces, then `split()` to get `words` array.

In [5]:
import re
text = re.sub(r'\W+', ' ', text)

words = text.split()
words[:10]

['befekadu',
 'mezgebu',
 'temere',
 'responsive',
 'web',
 'application',
 'using',
 'bootstrap',
 'and',
 'foundation']

In [15]:
# Length of extracted word sequences
maxlen = 10

# We sample a new sequence every `step` word
step = 3

# This holds our extracted sequences
sentences = []

# This holds the targets (the follow-up words)
next_words = []

for i in range(0, len(words) - maxlen, step):
    sentences.append(words[i: i + maxlen])
    next_words.append(words[i + maxlen])
print('Number of sequences:', len(sentences))


# List of unique words in the corpus
unique_words = sorted(list(set(words)))
print('Unique words:', len(unique_words))
# Dictionary mapping unique words to their index in `unique_words`
word_indices = dict((word, unique_words.index(word)) for word in unique_words)

print('Vectorization...')
x = np.zeros((len(sentences), maxlen))
y = np.zeros((len(sentences), len(unique_words)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, word in enumerate(sentence):
        x[i, t] = word_indices[word]
    y[i, word_indices[next_words[i]]] = 1
    
print("x shape: %s" % (str(x.shape)))
print("y shape: %s" % (str(y.shape)))

Number of sequences: 35451
Unique words: 6679
Vectorization...
x shape: (35451, 10)
y shape: (35451, 6679)


## Build network

In [16]:
model = models.Sequential()
model.add(layers.Embedding(len(unique_words), 32))
model.add(layers.LSTM(128))
model.add(layers.Dense(len(unique_words), activation='softmax'))
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 32)          213728    
_________________________________________________________________
lstm_4 (LSTM)                (None, 128)               82432     
_________________________________________________________________
dense_4 (Dense)              (None, 6679)              861591    
Total params: 1,157,751
Trainable params: 1,157,751
Non-trainable params: 0
_________________________________________________________________


## Train model and sample

Reweight the original probability distribution coming out of the model, and draw a word index from it 

In [0]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

Repeatedly train and generate text

In [19]:
import random
import sys

for epoch in range(1, 80):
    print('epoch', epoch)
    # Fit the model for 1 epoch on the available training data
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    # Select a text seed at random
    start_index = random.randint(0, len(words) - maxlen - 1)
    generated_text = words[start_index: start_index + maxlen]
    generated_text_str = ' '.join(generated_text)
    print('--- Generating with seed: "' + generated_text_str + '"')

    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text_str + " ")

        # We generate 40 words
        for i in range(40):
            sampled = np.zeros((1, maxlen))
            for t, word in enumerate(generated_text):
                sampled[0, t] = word_indices[word]

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_word = words[next_index]

            generated_text.append(next_word)
            generated_text = generated_text[1:]

            sys.stdout.write(next_word + " ")
            sys.stdout.flush()
        print()

epoch 1
Epoch 1/1
35451/35451 [==============================] - 7s 195us/step - loss: 3.9424
--- Generating with seed: "to be saved in mongodb by providing validation rules at"
------ temperature: 0.2
to be saved in mongodb by providing validation rules at external and the 8 this is the and and project 7 before external of external means plugins external wireframe figure frameworks a figure pagespeed mobile internally plugins it web user and 0 the templates is the the source constant css 
------ temperature: 0.5
to be saved in mongodb by providing validation rules at of code including view class syntactically the to 18 paper web user and the the was the two simplifies 17 were one a that which fix class translated the the project results plugins they components framework javascript they they without 
------ temperature: 1.0
to be saved in mongodb by providing validation rules at relative web user and external according the users as that a is the of for and also features bootstrap 1 pop

a fluid is resource syntactically file easiness popular external few the twitter the is web entire to to and which tablets including constant mobile figure abstract iii used pagespeed constant three the moreover 6 paper mobile the 
epoch 2
Epoch 1/1
35451/35451 [==============================] - 7s 201us/step - loss: 3.9154
--- Generating with seed: "url https www datical com blog 6 problems devops helps"
------ temperature: 0.2
url https www datical com blog 6 problems devops helps the of external resource megabyte is the about about about about external wireframe entire than external were the 1 popular rules to external technical is web about including it size pagespeed plugins external were the external of external 1 external 
------ temperature: 0.5
url https www datical com blog 6 problems devops helps of is illustrated class firefox pagespeed external of external were the according the in external were the 5 paper external moreover external as class the than com versions plugins 

syntactically websites forms external and external means images 
------ temperature: 1.0
url https www datical com blog 6 problems devops helps well in last syntactically is source the the the constant in 5 where including css in web video they they tested web frameworks latest syntactically fast in external and of is in a project be easiness pagespeed 17 with frameworks 
------ temperature: 1.2
url https www datical com blog 6 problems devops helps features the were fluid the moreover websites the region plugins performing external 1 the according the content class pagespeed and and of and website including it website is and 8 templates left 1 with conclude and an 1 that with 
epoch 3
Epoch 1/1
21632/35451 [=================>............] - ETA: 2s - loss: 3.6845

35451/35451 [==============================] - 7s 206us/step - loss: 3.8885
--- Generating with seed: "leads to a less error prone code this is most"
------ temperature: 0.2
leads to a less error prone code this is most plugins speed got 4 trend using and and external technical is the templates the was several was the improving external wireframe next next tool plugins 1 befekadu of external resource and speed plugins external as easiness with external april used 
------ temperature: 0.5
leads to a less error prone code this is most were the package to second 8 as is the and of website creating about websites april a was class like web user and external elements they they a px rules it line research plugins speed using for templates 3 resource 
------ temperature: 1.0
leads to a less error prone code this is most including optimizilla plugins pagespeed 1 paper web syntactically 3 package to of external the project and and project websites mobile optimize was one was of performing secon

plugins plugins cross below external shows external dom external view it the two internally 4 external and as for has project is example size pagespeed compared the 
epoch 4
Epoch 1/1
35451/35451 [==============================] - 7s 207us/step - loss: 3.8720
--- Generating with seed: "jsx is statically typed and mostly typesafe the quality of"
------ temperature: 0.2
jsx is statically typed and mostly typesafe the quality of external of images syntactically 4 popular project external were the also real a for the results plugins the this this this research mobile internally according the popular changing external download plugins computer tv components several technical frameworks they they they 
------ temperature: 0.5
jsx is statically typed and mostly typesafe the quality of they they they they they they used 35 plugins they they of is the for project the according the more footer experience device project well external technical trend the that the results smaller that they componen

...

35451/35451 [==============================] - 7s 207us/step - loss: 3.0427
--- Generating with seed: "that particular module the module of facebook s chatroom has"
------ temperature: 0.2
that particular module the module of facebook s chatroom has and websites they components they they they they they they they they they they they they they they they they they they they they they they they they they they they they they they they they they they they they 
------ temperature: 0.5
that particular module the module of facebook s chatroom has they they they they they they they they they they they they they they they they they they they they they they they they they they they they they they they they they they they they they they they they 
------ temperature: 1.0
that particular module the module of facebook s chatroom has they they they they they tv framework javascript class the project about with is used class project 3 frontend class as usage plugins including to changing the and moreo

features in pagespeed means external external web external 2 open 7 for at external web about were plugins the twitter package the syntactically web different performing 
epoch 78
Epoch 1/1
35451/35451 [==============================] - 7s 207us/step - loss: 3.0220
--- Generating with seed: "the opportunity to immediately see the results of their work"
------ temperature: 0.2
the opportunity to immediately see the results of their work 4 popular preparing speed versions plugins file resource web paper used pagespeed was multitude components including relative the easiness the it in result rem external by external as it and a rules preprocessor abstract apis folder performing external means window 
------ temperature: 0.5
the opportunity to immediately see the results of their work website structures engineering it illustrated class provided by computer external enterprise website was web css services external another means external 2 in as web got embedded class is the rules project an

simplifies popular source rules like web that structure class tv in websites external 38 device plugins external the content plugins external 1 images syntactically they firm illustrated class be syntactically above 3 creating web 1 was cli plugins syntactically computer 
------ temperature: 1.2
the opportunity to immediately see the results of their work is web 8 and was to and the of following and including two semantic figure two is on is the figure speed it search bootstrap chosen resource according the to used 3 that development web hypertext people the the mobile 
epoch 79
Epoch 1/1
30208/35451 [========================>.....] - ETA: 1s - loss: 2.9412

35451/35451 [==============================] - 7s 205us/step - loss: 3.0133
--- Generating with seed: "should not require any stored data to process the request"
------ temperature: 0.2
should not require any stored data to process the request syntactically the is the constant were the web different that website performing in result speed prepare external open 7 and of external view syntactically of external means external web external technical class folder mengistu power foundation and things framework is 
------ temperature: 0.5
should not require any stored data to process the request external relative syntactically in external external plugins external the that website website they components they they they they they they they they they they tv of web user many mengistu web 000 website including is foundation started pagespeed firefox pagespeed 
------ temperature: 1.0
should not require any stored data to process the request the 17 3 paper web got 1 and of language term creating 

folder are the video syntactically external external external external external of is external makes in websites of web user and 14 before external foundation the results programming in and by all external web 


**Generated text after 80 epochs**  
--- Generating with seed: "should not require any stored data to process the request"  
------ temperature: 0.2  
should not require any stored data to process the request syntactically the is the constant were the web different that website performing in result speed prepare external open 7 and of external view syntactically of external means external web external technical class folder mengistu power foundation and things framework is   
------ temperature: 0.5  
should not require any stored data to process the request external relative syntactically in external external plugins external the that website website they components they they they they they they they they they they tv of web user many mengistu web 000 website including is foundation started pagespeed firefox pagespeed   
------ temperature: 1.0  
should not require any stored data to process the request the 17 3 paper web got 1 and of language term creating websites 4 another view website external compiler define a makes in web to cli the in 4 in world the and web results templates developers wheel performing good   
------ temperature: 1.2  
should not require any stored data to process the request the a makes external in external class folder are the video syntactically external external external external external of is external makes in websites of web user and 14 before external foundation the results programming in and by all external web   

___
It seems that generated technical text did not make good sense. Also the theses topics vary a lot. Maybe I should have gone for the philosophy related theses instead, then the confusing generated text will sound smart. 